In [1]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime
import time
import talib as ta

In [2]:
def get_swingtrade_stocklist():
  df = pd.read_csv('swingtradestocks_list.csv')
  return df

In [3]:
df_reverse = pd.read_csv('lunar_2022.csv')

In [4]:
def download_stock(tckr, start_date, end_date):
  start = start_date
  end = end_date
  symbol = tckr
  symbol = symbol+'.NS'
  stock_data = yf.download(symbol, start=start, end=end,rounding=True)
  if(len(stock_data)<1):
    stock_data = pd.DataFrame()
  return stock_data

In [5]:
#x=datetime.strptime(df_reverse.values[0][0], '%Y-%m-%d').date()

In [6]:
def trend_reversal(): # this will give, if the trend changes tomorrow or not and also predict the trend reversal date
    current_date = date.today()
    previous_date = current_date - timedelta(days = 1)
    next_day = current_date + timedelta(days =1)

    # is the trend reversing tommorow?
    if(df_reverse['reversal_dates'].str.contains(str(current_date)).any()
        | df_reverse['reversal_dates'].str.contains(str(previous_date)).any() | df_reverse['reversal_dates'].str.contains(str(next_day)).any()):
        reverse_trend = 'yes'
    else:
        reverse_trend = 'no'
    
    #current trend will end on the following date..  
    # will give next reversal date, if the target is not achieved then square it off before this date
    # next trend reversal dates

    trend_end_date = datetime.strptime(df_reverse.loc[df_reverse['reversal_dates']>=str(current_date)].values[0][0], '%Y-%m-%d').date()
    return reverse_trend, trend_end_date   

In [7]:
def decide_position(symbol):
    current_date = date.today()

    # previous moon date finder
    previous_trend_start_date = df_reverse.loc[df_reverse['reversal_dates']<=str(current_date)].values[-1][0]
    previous_trend_start_date=datetime.strptime(previous_trend_start_date, '%Y-%m-%d').date()
    
    # get closing price of that day
    
    stock_data_prev_tren_start_day = download_stock(symbol,previous_trend_start_date,previous_trend_start_date + timedelta(days=1))
    if(len(stock_data_prev_tren_start_day)<1):
        stock_data_prev_tren_start_day = download_stock(symbol,previous_trend_start_date,previous_trend_start_date + timedelta(days=3))
   
    # Get EMA on previous moon day
    data_prev_moon = download_stock(symbol,previous_trend_start_date - timedelta(days = 20),previous_trend_start_date + timedelta(days=1))
   
    if(len(data_prev_moon)>0):
    # calculate EMA and decide the direction
        ema_prev_moon = ta.EMA(data_prev_moon.Close, timeperiod=4)
    

    # Calculate EMA on current day
    data = download_stock(symbol,current_date - timedelta(days = 20),current_date+timedelta(days=1))
    ltp = data['Close'][-1]
    
    if(len(data)>0):
        ema_current = ta.EMA(data.Close, timeperiod=7)

    trend_rev = trend_reversal() 

    # check if the current trend is an upward trend or downward trend
    #if(current close  > previous moon close)and ema on prev moon(+ or - 1) < ema current date then short it, if trend reversal is yes
    if((ltp > stock_data_prev_tren_start_day['Close'][-1]) & (ema_prev_moon[-1] < ema_current[-1])):
        signal = 'buy' # meaning current trend is upward and the next trend is going to be downward
    if((ltp < stock_data_prev_tren_start_day['Close'][-1]) & (ema_prev_moon[-1] > ema_current[-1])):
        signal = 'sell'  # meaning current trend is downward and the next trend is going to be upward
    else:
        signal = 'none'

    return signal, str(trend_rev[1]) # gives signal and the next trend reversal date

In [8]:
def Gann_prediction(LTP):
  
  # find sq root of the ltp
  sq_rt_ltp = math.sqrt(LTP)
  #ceil the sq rt
  cieled_sq_rt = math.ceil(sq_rt_ltp)
  #get starting number for iteration
  lowest_num = cieled_sq_rt-2
  block = [lowest_num, lowest_num+1, lowest_num+2]
  
  for i in (block):
    centre_1 = math.pow(i+0.125, 2)
    centre_2 = math.pow(i+0.125*2, 2)
    centre_3 = math.pow(i+0.125*3, 2)
    centre_4 = math.pow(i+0.125*4, 2)
    centre_5 = math.pow(i+0.125*5, 2)
    centre_6 = math.pow(i+0.125*6, 2)
    centre_7 = math.pow(i+0.125*7, 2)
    centre_8 = math.pow(i+0.125*8, 2)
    if(i==block[0]):
      level_1 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[1]):
      level_2 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[2]):
      level_3 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
  
  #merge list
  Gann_list = level_1 + level_2 + level_3


  insrt_loc = 0  
  for j in range(len(Gann_list)):
    if(Gann_list[j] < LTP < Gann_list[j+1]):
      insrt_loc = j+1
      Gann_list.insert(insrt_loc, LTP)
      break

  Resistance_1, Resistance_2, Resistance_3, Resistance_4 = (Gann_list[insrt_loc+2], Gann_list[insrt_loc+3],
                                                                       Gann_list[insrt_loc+4], Gann_list[insrt_loc+5])

  Support_1, Support_2, Support_3, Support_4 = (Gann_list[insrt_loc-2], Gann_list[insrt_loc-3],
                                                                       Gann_list[insrt_loc-4], Gann_list[insrt_loc-5])

  buy_above = Gann_list[insrt_loc+1]
  c1 = 0.99995
  sell_target_1, sell_target_2, sell_target_3, sell_target_4 =  (round(c1*Resistance_1,2), round(c1*Resistance_2,2), 
                                                                              round(1*Resistance_3,2), round(c1*Resistance_4,2))
  sl_long = Gann_list[insrt_loc-1]
  sell_below = Gann_list[insrt_loc-1]
  c2 = 1.00005
  buy_target_1, buy_target_2, buy_target_3, buy_target_4 = (round(c2*Support_1,2), round(c2*Support_2,2), round(c2*Support_3,2), 
                                                                        round(c2*Support_4,2))
  sl_short = Gann_list[insrt_loc+1]
  
  return (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
          sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)
  

In [9]:
def profit_percentage(prediction_list_df): # this function will use multiple levls of filtering
    
    stock_tckr = prediction_list_df['stock_tckr']
    buy_price =  prediction_list_df['buy_above'] #buying price for long trade
    sell_price = prediction_list_df['sell_below'] # selling price for short trade
    sl_long = prediction_list_df['sl_long']
    sl_short = prediction_list_df['sl_short']

    # Percentage profit with target 1 achieved for long trade
    pct_profit_long_tgt_1 = (prediction_list_df['sell_target_1'] - buy_price) * 100/ buy_price
    # Percentage profit with target 2 achieved for long trade
    pct_profit_long_tgt_2 = (prediction_list_df['sell_target_2'] - buy_price) * 100/ buy_price
    # Percentage profit with target 3 achieved for long trade
    pct_profit_long_tgt_3 = (prediction_list_df['sell_target_3'] - buy_price) * 100/ buy_price
    # Percentage profit with target 4 achieved for long trade
    pct_profit_long_tgt_4 = (prediction_list_df['sell_target_4'] - buy_price) * 100/ buy_price

    # Percentage profit with target 1 achieved for short trade
    pct_profit_short_tgt_1 = (sell_price - prediction_list_df['buy_target_1']) *100/ prediction_list_df['buy_target_1']
    # Percentage profit with target 2 achieved for short trade
    pct_profit_short_tgt_2 = (sell_price - prediction_list_df['buy_target_2']) *100/ prediction_list_df['buy_target_2']
    # Percentage profit with target 3 achieved for short trade
    pct_profit_short_tgt_3 = (sell_price - prediction_list_df['buy_target_3']) *100/ prediction_list_df['buy_target_3']
    # Percentage profit with target 4 achieved for short trade
    pct_profit_short_tgt_4 = (sell_price - prediction_list_df['buy_target_4']) *100/ prediction_list_df['buy_target_4']
    
    # check stop loss vs pct profit
    # Risk reward percentage: long trade vs stop loss
    loss_pct_against_long_trade = (buy_price - sl_long) * 100/ buy_price
    # Risk reward percentage: short trade vs stop loss
    loss_pct_against_short_trade = (sl_short - sell_price) * 100/ sl_short

    return (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
            pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
            loss_pct_against_long_trade, loss_pct_against_short_trade)

In [10]:
def stock_filtering(long_profit, long_loss, short_profit, short_loss):
    if((long_profit>(2*long_loss) or short_profit>(2*short_loss)) and (long_profit>2 or short_profit>2)):
        return "good_stock"
    else:
        return "remove_stock"


In [11]:
# Main 
swingtrade_stock_list = get_swingtrade_stocklist()
prediction_list =[]
risk_rewards = []
current_date = date.today() + timedelta(days=1) # this adjustment is required because python does not include last date of the range.
start_date = date.today() - timedelta(days=1) # previous 1 day from today


In [12]:
# Gann prediction algorithm
for i in range(len(swingtrade_stock_list)-1):
#for i in range(0,10): # test with 3 stocks
  swingtrade_stock_tckr = swingtrade_stock_list['Swingtrade'][i] # get ticker symbol
  swingtrade_stock_data = download_stock(swingtrade_stock_tckr, start_date ,current_date) # download data from yfinance 
  if(len(swingtrade_stock_data)>0 and not(pd.isnull(swingtrade_stock_data.iloc[0,3]))):
    current_day_close = round(swingtrade_stock_data.values[-1][3],2)

    (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)= Gann_prediction(current_day_close)
          
    prediction_list.append([swingtrade_stock_tckr,buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4])

prediction_list_df = pd.DataFrame(prediction_list, columns=['stock_tckr','buy_above', 'sl_long', 'sell_target_1', 'sell_target_2', 
        'sell_target_3', 'sell_target_4', 'sell_below', 'sl_short', 'buy_target_1', 'buy_target_2', 'buy_target_3', 'buy_target_4'])



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HEI.NS: No d

In [13]:
risk_rewards =[]

In [14]:
#filtering low profit stocks
#for i in range(0,100): # test with 3 stocks
for i in range(len(prediction_list_df)):
  (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade) = profit_percentage(prediction_list_df.iloc[i])

  risk_rewards.append([stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade])

risk_rewards_df = pd.DataFrame(risk_rewards, columns = ['stock_tckr', 'pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3',
 'pct_profit_long_tgt_4', 'pct_profit_short_tgt_1', 'pct_profit_short_tgt_2', 'pct_profit_short_tgt_3', 'pct_profit_short_tgt_4', 
 'loss_pct_against_long_trade', 'loss_pct_against_short_trade'])

# Merge both data frames, Gann predictions and risk rewards for evaluation
reduced_stocks_df = pd.merge(prediction_list_df, risk_rewards_df,  on = 'stock_tckr')

# Filter elimination criteria
reduced_stocks_df['stock_filter'] = reduced_stocks_df.apply(lambda x: stock_filtering(x.pct_profit_long_tgt_2, x.loss_pct_against_long_trade, x.pct_profit_short_tgt_2,
                                                        x.loss_pct_against_short_trade), axis=1)
reduced_stocks_df = reduced_stocks_df[reduced_stocks_df['stock_filter']!='remove_stock'] # drop low profit stocks
reduced_stocks_df=reduced_stocks_df.reset_index(drop=True) #reset index


Decide direction and predict trend reversal date

In [15]:
# Swing trades
signal_list =[]
for i in range(len(reduced_stocks_df)):
  #for i in range(0,10):
  swingtrade_stock_tckr = reduced_stocks_df['stock_tckr'][i] # get ticker symbol

  # call decide postion and trend reversal function
  signal, trend_reversal_date = decide_position(swingtrade_stock_tckr)
  signal_list.append([swingtrade_stock_tckr, signal, trend_reversal_date])

#create df of signal direction
signal_list_df = pd.DataFrame(signal_list, columns =['stock_tckr', 'signal', 'trend_reversal_date'])

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SJVN.NS: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HINDZINC.NS: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SAIL.NS: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%*************

In [16]:
# create a df to backtest using actual data against predicted data
final_stocks_for_tomorrow = pd.merge(reduced_stocks_df,signal_list_df, on = 'stock_tckr')

In [17]:
final_stocks_for_tomorrow.rename(columns={'stock_tckr': 'Stock symbol', 'buy_above': 'Buy above', 
'sl_long': 'Stop loss', 'sell_target_1':'Sell target 1','sell_target_2':'Sell target 2','sell_target_3':'Sell target 3',
'sell_target_4':'Sell target 4', 'pct_profit_long_tgt_4':'Max Gain(%)', 'trend_reversal_date':'Trend reversal date'}, inplace=True)

In [18]:
final_stocks_for_tomorrow.drop(['sell_below', 'sl_short', 'buy_target_1', 'buy_target_2', 'buy_target_3', 'buy_target_4', 
'pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3','pct_profit_short_tgt_1', 'pct_profit_short_tgt_2',
       'pct_profit_short_tgt_3', 'pct_profit_short_tgt_4', 'loss_pct_against_long_trade', 'loss_pct_against_short_trade','stock_filter'], 
       axis=1, inplace=True)

In [19]:
tomorrow=date.today() + timedelta(days=1)

In [20]:
final_stocks_for_tomorrow.insert(loc = 0,
          column = 'Date',
          value = tomorrow)

In [21]:
final_stocks_for_tomorrow = final_stocks_for_tomorrow[final_stocks_for_tomorrow.signal == 'buy']

In [22]:
final_stocks_for_tomorrow.to_csv('Predictions/swingtrade_predictions_for_tomorrow.csv')